## 使用真正的 T5-Chinese 模型


In [4]:
import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.auto import tqdm

# 1. 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 加载数据
print("正在加载数据集...")
dataset = load_dataset("Helsinki-NLP/opus-100", "en-zh", split="test") 
test_data = dataset.select(range(2000))
english_inputs = [item['translation']['en'] for item in test_data]
chinese_references = [item['translation']['zh'] for item in test_data]

# 3. 加载实验指定的模型 (utrobinmv)
# -------------------------------------------------------
model_name = "utrobinmv/t5_translate_en_ru_zh_small_1024"
print(f"正在加载指定模型: {model_name} ...")

# 必须加 use_fast=False 解决 Windows 下的兼容性报错
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# 4. 【关键修正】修改 T5 的指令前缀
# -------------------------------------------------------
# 之前用 "translate English to Chinese: " 导致了输出俄语
# 这次我们要用这个模型能听懂的指令：
T5_PREFIX = "translate to zh: "

BATCH_SIZE = 8  # 如果显存报错，请改为 8
MAX_LENGTH = 128

# 5. 执行推理
predictions = []
print(f"\n开始推理 (使用修正后的指令 '{T5_PREFIX}')...")

for i in tqdm(range(0, len(english_inputs), BATCH_SIZE), desc="翻译进度"):
    batch_inputs = english_inputs[i:i + BATCH_SIZE]
    
    # 将指令拼接到输入文本前
    inputs_with_prefix = [T5_PREFIX + text for text in batch_inputs]
    
    tokenized_inputs = tokenizer(inputs_with_prefix, 
                                 return_tensors="pt", 
                                 padding=True, 
                                 truncation=True, 
                                 max_length=MAX_LENGTH)
    
    input_ids = tokenized_inputs.input_ids.to(device)
    attention_mask = tokenized_inputs.attention_mask.to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids, 
            attention_mask=attention_mask, 
            max_length=MAX_LENGTH, 
            num_beams=4,
            early_stopping=True
        )
    
    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions.extend(decoded_preds)

# 6. 评估
print("\n--- 正在计算评估指标 ---")
formatted_references = [[ref] for ref in chinese_references]
bleu_metric = evaluate.load("sacrebleu")
bleu_results = bleu_metric.compute(predictions=predictions, references=formatted_references)
print(f"\n✅ BLEU Score: {bleu_results['score']:.2f}")

print("\n--- 结果抽查 ---")
print(f"【原文】: {english_inputs[0]}")
print(f"【预测】: {predictions[0]}")

正在加载数据集...
正在加载指定模型: utrobinmv/t5_translate_en_ru_zh_small_1024 ...

开始推理 (使用修正后的指令 'translate to zh: ')...


翻译进度: 100%|██████████| 250/250 [09:33<00:00,  2.30s/it]



--- 正在计算评估指标 ---

✅ BLEU Score: 13.09

--- 结果抽查 ---
【原文】: The Global Programme of Action Coordination Office, with the financial support of Belgium, is currently assisting Egypt, Nigeria, United Republic of Tanzania, Sri Lanka and Yemen to develop pilot national programmes of action for the protection of the marine environment from land-based activities.
【预测】: 全球行动纲领协调办公室在比利时的财政支持下,目前正在协助埃及、尼日利亚、坦桑尼亚联合共和国、斯里兰卡和也门制定保护海洋环境免受陆上活动影响的试点国家行动方案。


In [24]:
import random


print("-" * 50)
print("📊 开始执行独立评估模块")
print("-" * 50)


# 随机抽查 20 条结果
# -------------------------------------------------------
SAMPLE_NUM = 20
total_count = len(predictions)

print("\n" + "=" * 50)
print(f"🧐 随机抽查 {SAMPLE_NUM} 条翻译结果 (总数: {total_count})")
print("=" * 50)

# 使用 random.sample 进行无放回抽样
# 这样每次运行这个代码块，您都能看到不同的 20 句话
random_indices = random.sample(range(total_count), min(SAMPLE_NUM, total_count))

for i, idx in enumerate(random_indices):
    print(f"\n📝 [样本 #{i+1} / 索引 {idx}]")
    print(f"🔵 原文 (EN): {english_inputs[idx]}")
    print(f"🟢 参考 (ZH): {chinese_references[idx]}")
    print(f"🔴 预测 (T5): {predictions[idx]}")
    print("-" * 30)

--------------------------------------------------
📊 开始执行独立评估模块
--------------------------------------------------

🧐 随机抽查 20 条翻译结果 (总数: 2000)

📝 [样本 #1 / 索引 1545]
🔵 原文 (EN): ...and then I start.
🟢 参考 (ZH): ...我開始了
🔴 预测 (T5): 然后我开始。
------------------------------

📝 [样本 #2 / 索引 18]
🔵 原文 (EN): The movement of equipment by road from Port Sudan to Darfur, some 1,400 miles, currently takes an average of seven weeks.
🟢 参考 (ZH): 从苏丹港到达尔富尔的路程大约1 400英里，目前装备平均需要7周时间才能运到。
🔴 预测 (T5): 目前,从苏丹港到达尔富尔大约1 400英里的公路运输设备平均需要7周。
------------------------------

📝 [样本 #3 / 索引 245]
🔵 原文 (EN): I will forward to the Council my conclusions based on the report of this mission as an addendum to the present report for its consideration.
🟢 参考 (ZH): 我将根据核查团的报告作出结论，作为本报告补编提交安理会审议。
🔴 预测 (T5): 我将根据本特派团的报告向安理会提出我的结论,作为本报告增编供其审议。
------------------------------

📝 [样本 #4 / 索引 801]
🔵 原文 (EN): In this regard, it encourages the Secretary-General to take steps to employ staff in the Department of Peacekeeping Operations and th

In [25]:
import pandas as pd
import os

# 1. 检查数据完整性
# -------------------------------------------------------
if len(predictions) != len(english_inputs) or len(predictions) != len(chinese_references):
    print(f"⚠️ 警告：数据长度不一致！")
    print(f"原文: {len(english_inputs)}, 参考: {len(chinese_references)}, 预测: {len(predictions)}")
    # 如果长度不一致，取最小值截断，防止报错
    min_len = min(len(english_inputs), len(chinese_references), len(predictions))
    data = {
        "English Input (原文)": english_inputs[:min_len],
        "Chinese Reference (参考答案)": chinese_references[:min_len],
        "Model Prediction (模型预测)": predictions[:min_len]
    }
else:
    # 2. 准备数据
    # -------------------------------------------------------
    data = {
        "English Input (原文)": english_inputs,
        "Chinese Reference (参考答案)": chinese_references,
        "Model Prediction (模型预测)": predictions
    }

# 3. 创建 DataFrame
# -------------------------------------------------------
df = pd.DataFrame(data)

# 4. 导出为 CSV 文件
# -------------------------------------------------------
# 文件名
output_filename = "t5_translation_results_2000.csv"

# encoding='utf-8-sig' 是关键！
# 它会添加 BOM 头，确保您在 Windows 上用 Excel 打开时中文不会乱码
df.to_csv(output_filename, index=False, encoding='utf-8-sig')

# 5. 获取文件绝对路径并打印
# -------------------------------------------------------
current_dir = os.getcwd()
full_path = os.path.join(current_dir, output_filename)

print(f"✅ 导出成功！包含 {len(df)} 条数据。")
print(f"📂 文件保存在: {full_path}")
print("\n--- 前 5 行预览 ---")
print(df.head())

✅ 导出成功！包含 2000 条数据。
📂 文件保存在: d:\课程资料\机器学习实践\1119-1126鏈哄櫒瀛︿範缁煎悎瀹炶返11-12\t5_translation_results_2000.csv

--- 前 5 行预览 ---
                                  English Input (原文)  \
0  The Global Programme of Action Coordination Of...   
1  The forum produced business contracts amountin...   
2  But, during the 1990’s, and for reasons that a...   
3  Not since the inception of its mandate in 1968...   
4  “6. Also welcomes the aspiration of the countr...   

                            Chinese Reference (参考答案)  \
0  9. 《全球行动纲领》协调处得到比利时的财政支持，目前正帮助埃及、尼日利亚、坦桑尼亚联合共和...   
1                该论坛达成的亚洲和非洲国家私营公司间的商业合同超过了2,400万美元。   
2  艾滋病来势汹汹及其传染速度让很多南非人意想不到。像发达国家一样，艾滋病毒感染最初集中在血友病...   
3  大概自1968年委员会开始执行其任务以来，委员会从未遇到被占领土内的巴勒斯坦人和其他阿拉伯人...   
4  “6.　又欢迎经济转型国家渴望进一步开展区域、次区域和区域间合作，请联合国系统加强同其成员包...   

                             Model Prediction (模型预测)  
0  全球行动纲领协调办公室在比利时的财政支持下,目前正在协助埃及、尼日利亚、坦桑尼亚联合共和国、...  
1                    论坛在亚洲和非洲私营公司之间签订了2400多万美元的商业合同。  
2  但是,在20世纪90年代期间,由于仍然被明确理解的原因,病毒爆炸到普通人群中